In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import json, os, logging, random, copy, shutil
import seaborn as sns, pandas as pd, numpy as np, nibabel as nib
from ipywidgets import interact, IntSlider, Select, HBox
from utilities import apply_windowing, resize_to_user_resolution
from dicom_tools import DicomToolbox   


In [ ]:
# get environment variables
import os

# Replace 'YOUR_ENV_VAR' with the name of your environment variable
env_var_value = os.getenv('nnUNet_raw')

print(env_var_value)

In [18]:
from tqdm.notebook import tqdm
from dicom_tools import DicomTools   
import logging, os, json, random, copy, shutil, h5py
import matplotlib.pyplot as plt
import numpy as np, nibabel as nib
from utilities import apply_windowing, resize_to_user_resolution
from typing import Tuple

from batchgenerators.utilities.file_and_folder_operations import save_json, join

params = {
    'DATA_DIRECTORY':  "/home/ivazquez/Data/anonymized/neck_cta", #, "/home/ivazquez/Documents/Data/liver",
    'PREPROCESSED_DATA_DIRECTORY': "/home/ivazquez/Repos/nnUNet/preprocessed_data/preprocessed_hdf5_files/preprocessed_Dataset001_NeckCTA.h5",
    'RESAMPLING': {'apply': True, 'resolution': (1.0, 1.0, 1.0)},
    'LABELS': ["common_carotid_artery_left"],
    'LABEL_GROUPS': None, 
    'DATASET_DESCRIPTION': {
        'id': 1,
        'name': 'NeckCTA',
        'description': 'Neck CTA dataset',
        'reference': 'HMH'
    },
    'TEST_SET_SIZE': 2,
    'LEAVE_OUT': None,
    'INPUT_MODALITIES': ['ct'],
    'OUTPUT_MODALITIES': ['rtstruct']}

class Preprocessing(DicomTools):
    
    '''
        Created by Ivan Vazquez in collaboration with Ramiro Pino
    '''
    
    def __init__(self, params):
        self.__start_logger()
        self.params = params
        super().__init__(patient_data_directory=params['DATA_DIRECTORY'])
        self.expected_data = params['INPUT_MODALITIES'] + params['OUTPUT_MODALITIES']
        self.labels = params['LABELS']
        self.echo_level = 0
        self.minimum_ct_value = -1024
        self.uniform_slice_thickness = False # forces uniform slice thickness for CT during parsing
        self.preprocessed_data_dir = params['PREPROCESSED_DATA_DIRECTORY']
        self.raw_data_dir = os.getenv('nnUNet_raw') # assumet that nnUNet environment variable is set and defined as nnUNet_raw
        self.preprocessed_data_fldr = os.getenv('nnUNet_preprocessed') 
        self.__data_id = f'Dataset{params["DATASET_DESCRIPTION"]["id"]:03}_{params["DATASET_DESCRIPTION"]["name"]}'
        
    def __start_logger(self):
        if not os.path.exists('logs'):
            os.makedirs('logs')
        log_file = os.path.join('logs','data_preparation.log')
        with open(log_file, 'w') as f:
            pass
        logging.basicConfig(filename=log_file, filemode='w',
                            level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
        
    def get_data_insight(self):
        all_pat_ids = self.identify_patient_files()
        self.data_insight = {p:{} for p in all_pat_ids}
        
        progress_bar = tqdm(all_pat_ids, desc="")
        
        for p in progress_bar:
            
            try:
                progress_bar.set_description(f'Getting data insight for pat-{p}')
                
                self.reset()
                
                self.run_initial_check(p)
                
                # Handle input modalities
                self.data_insight[p]['input_modalities'] = {m:0 for m in self.params['INPUT_MODALITIES']}
                                
                for i in self.params['INPUT_MODALITIES']:
                    if i == 'ct' and 'ct' in self.dicom_files:
                        self.ct = self.parse_ct_study_files(self.dicom_files['ct'])
                        self.data_insight[p]['input_modalities']['ct']={'shape': [i for i in self.ct.data.shape], 
                                                                        'min': self.ct.data.min(),
                                                                        'max': self.ct.data.max(),
                                                                        'dx': self.ct.coordinates.dx,
                                                                        'dy': self.ct.coordinates.dy,
                                                                        'dz': [i for i in self.ct.coordinates.dz],
                                                                        'position':self.ct.patient_position}
                    elif i == 'ct' and 'ct' not in self.dicom_files:
                        logging.warning(f"CT not available for patient {p}")
                        self.data_insight[p]['input_modalities']['ct'] = 'CT not available'
                    
                # Handle output labels (structures)     
                self.contours = self.parse_structure_files(files=self.dicom_files['structures'], names_only=True)
                self.data_insight[p]['labels'] = self.find_structures(p, contours = self.labels)
                self.data_insight[p]['number_of_structures'] = len(self.contours)
                self.data_insight[p]['structures_available'] = sorted([c for c in self.contours])
                
            except Exception as e:
                logging.error(f"Error in patient {p}: {e}")
                self.data_insight[p]['error'] = str(e)
                    
        with open(os.path.join(self.raw_data_dir,f'data_insight_{self.__data_id}.json'), 'w') as f:
            json.dump(self.data_insight, f, indent=4)
    
    def find_structures(self, pat_id, contours = [], desperate_mode = True):
                      
        # prepare dictionary with desired contours
        assert type(contours) == type([]) or type(contours) == type(()), "Contours must be a list or tuple"
        contours = {c:0 for c in self.labels} if contours == [] else {c:0 for c in contours}
                
        # Load the variations for contour names
        with open(os.path.join("assets", "structure_name_variations.json"),"r") as f:
            contour_variations = json.load(f)
        
        # items added to the name of contours that make it difficult to identify them
        with open(os.path.join("assets", "junk_symbols.json"),"r") as f:
            junk_symbols = json.load(f)['oar']

        # Grab available contours for current patients
        if hasattr(self, 'contours'):
            available_contours = self.contours if type(self.contours) == type([]) else self.contour.keys() 
        else:
            self.run_initial_check(pat_id)
            self.parse_structure_files(names_only = True)
            available_contours = self.contours
        
        lr_variations = contour_variations["sides"] # variations for left and right
        links = ['_','-'," ",""] # types of links used for word elements in contour names
                
        for c in contours:  

            side = lr_variations["left"]  if "left" in c else lr_variations["right"]  if "right" in c else [""]
            gland = contour_variations["gland"] + [""] if 'gland' in c else [""] 
            links2 = links if 'gland' in c else [""] # links to attach gland to name 
            k = '_'.join([kk for kk in c.split('_') if kk not in ['left', 'right', 'gland']]) 
              
            # generate name variations
            name_variations = [f"{n}{l1}{g}{l2}{s}" for n in contour_variations[k] for l1 in links for l2 in links2 for s in side for g in gland]
            name_variations += [f"{g}{l1}{n}{l2}{s}" for n in contour_variations[k] for l1 in links for l2 in links2 for s in side for g in gland]
            name_variations += [f"{s}{l1}{n}{l2}{g}" for n in contour_variations[k] for l1 in links for l2 in links2 for s in side for g in gland]
            name_variations += [f"{s}{l1}{g}{l2}{n}" for n in contour_variations[k] for l1 in links for l2 in links2 for s in side for g in gland]
                            
            for v in name_variations:

                if contours[c] != 0: break

                for n in available_contours:
                    if n == v: contours[c] = n.strip()

                for n in available_contours:
                    if contours[c] == 0:
                        n_new = "".join([i for i in n if not i.isdigit()])
                        if n_new.strip() == v: contours[c] = n.strip()

                for n in available_contours:
                    if contours[c] == 0:
                        n_new = "".join([i for i in n.strip() if i not in ['_','/','.','-']])
                        n_new = "".join([i for i in n_new if not i.isdigit()])
                        if n_new.strip() == v: contours[c]= n.strip()
                        
                for n in available_contours:
                    if desperate_mode and contours[c] == 0:
                        for item in junk_symbols:
                            n_new = n.strip().replace(item,"")
                            n_new = "".join([i for i in n_new.strip() if i not in ['_','/','.','-']])
                            n_new = "".join([i for i in n_new if not i.isdigit()])
                            if n_new.strip() == v: contours[c] = n.strip()    
                               
        return contours
    
    def prepare_data(self, data_insight_file_path= None):
        
        self.prepare_dataset_folders()
        
        if data_insight_file_path is not None:
            self.data_insight = json.load(open(data_insight_file_path, 'r'))
        else:
            self.get_data_insight()
            
        parent_dir = os.path.join(self.preprocessed_data_fldr, 'preprocessed_hdf5_files')
        if not os.path.exists(parent_dir): os.makedirs(parent_dir)
            
        if self.preprocessed_data_dir is not None and os.path.exists(self.preprocessed_data_dir):
            preprocess_data = False
        elif self.preprocessed_data_dir is not None and not os.path.exists(self.preprocessed_data_dir):
            preprocess_data = True
            logging.warning(f"Preprocessed data file not found at {self.preprocessed_data_dir}."  + 
                            "Will create new preprocessed data file")
            self.preprocessed_data_dir = os.path.join(parent_dir, f'preprocessed_{self.__data_id}.h5')
        elif self.preprocessed_data_dir is None:
            preprocess_data = True
            self.preprocessed_data_dir = os.path.join(parent_dir, f'preprocessed_{self.__data_id}.h5')
    
        if preprocess_data:
                                
            with h5py.File(self.preprocessed_data_dir, 'w') as f:
                
                progress_bar = tqdm([p for p in self.data_insight], desc="")
                
                for p in progress_bar:
                    
                    progress_bar.set_description(f'Preprocessing data for pat-{p}')
                    
                    patient_info = self.data_insight[p]
                    if 'error' in patient_info: continue 
                    if not all([patient_info['labels'][l] for l in patient_info['labels']]): continue
                                    
                    self.reset()
                    self.run_initial_check(p)  
                    
                    for i in self.params['INPUT_MODALITIES']:
                        
                        files = self.dicom_files[i]
                        
                        if i == 'ct':
                            mod = self.parse_ct_study_files(files)
                            
                        # TODO: addapt to other modalities 
                                            
                        # resample data here
                        orig_coords = copy.deepcopy(mod.coordinates) 
                        new_res = self.params['RESAMPLING']['resolution']
                        
                        mod_data, coords = resize_to_user_resolution(mod.data, orig_coords, new_res, fill_value=mod.data.min()) if self.params['RESAMPLING']['apply'] else (mod.data, orig_coords)
                    
                        f.create_dataset(f'{p}/modalities/{i}', data=mod_data, compression='lzf')
                        f.create_dataset(f'{p}/coordinates/x', data=coords.x)
                        f.create_dataset(f'{p}/coordinates/y', data=coords.y)
                        f.create_dataset(f'{p}/coordinates/z', data=coords.z)
                        f.create_dataset(f'{p}/coordinates/dx', data=coords.dx)
                        f.create_dataset(f'{p}/coordinates/dy', data=coords.dy)
                        f.create_dataset(f'{p}/coordinates/dz', data=coords.dz)
                                
                    for label in self.labels:
                        contour_name = patient_info['labels'][label]
                        contour = self.parse_structure_files(files=sorted(self.dicom_files['structures']), patient_id=p, mask_names=contour_name, resolution='ct')
                        mask, _ = resize_to_user_resolution(contour[contour_name].data, orig_coords, new_res, fill_value=0) if self.params['RESAMPLING']['apply'] else (contour[label].data, None)
                        f.create_dataset(f'{p}/labels/{label}', data=np.round(mask).astype(np.uint8))
        
        self.create_data_breakdowns(self.preprocessed_data_dir)
                        
    def save_as_nifti(self, volume, voxel_spacing, output_path, **kwargs):
        
        # grab header information from kwargs
        modality = kwargs.get('modality', 'CT')
        
        # header = nifti_img.header

        if not isinstance(voxel_spacing, (list, tuple)) or len(voxel_spacing) != 3:
            raise ValueError("voxel_spacing must be a list or tuple with 3 elements.")

        affine = np.eye(4)
        affine[0, 0] = voxel_spacing[0]  # Spacing in x
        affine[1, 1] = voxel_spacing[1]  # Spacing in y
        affine[2, 2] = voxel_spacing[2]  # Spacing in z

        nifti_image = nib.Nifti1Image(volume, affine=affine)
        
        header = nifti_image.header
        header.set_xyzt_units('mm', 'sec')
        header['qform_code'] = 1
        header['descrip'] = modality
        header['pixdim'] = np.array([1.0, voxel_spacing[0], voxel_spacing[1], voxel_spacing[2], 1.0, 1.0, 1.0, 1.0])
        
        nib.save(nifti_image, output_path)       
    
    def prepare_dataset_folders(self):
        
        self.dataset_path = os.path.join(self.raw_data_dir, self.__data_id) # depends on the nnUNet environment variable
        if os.path.exists(self.dataset_path ): shutil.rmtree(self.dataset_path )
        os.makedirs(self.dataset_path , exist_ok=True)  
        
        for dir_name in ['imagesTr', 'imagesTs', 'labelsTr', 'labelsTs']:
            os.makedirs(os.path.join(self.dataset_path , dir_name), exist_ok=True)
    
    def create_data_breakdowns(self, data_path):
        
        with h5py.File(data_path, 'r') as f:
            all_data_ids = list(f.keys())
        
        if self.params['LEAVE_OUT'] is not None: all_data_ids = [int(pat) for pat in all_data_ids if int(pat) not in self.params['LEAVE_OUT']]

        self.test_set = random.sample(all_data_ids, self.params['TEST_SET_SIZE'])
        self.train_set = [n for n in all_data_ids if n not in self.test_set]
        
        if self.params['LABEL_GROUPS'] is None:
            label_groups = {k:{'labels':[k], 'id':n+1} for n,k in enumerate(self.labels)}
        else:
            label_groups = self.params['LABEL_GROUPS']
            label_groups = {k:{'labels':[l for l in v], 'id':n+1} for n,(k,v) in enumerate(label_groups.items())}
                            
        with h5py.File(data_path, 'r') as hf:
            
            self.__save_data(hf, 'Tr', label_groups)
        
            self.__save_data(hf, 'Ts', label_groups)  
            
        
        
        
        
        # output_dir = '../raw_data/Dataset001_NeckCTA'
        # channel_names = {0: 'CT'}
        # labels = {'common_carotid_artery_left': 1,
        #         'background': 0}
        # num_training_cases = len(os.listdir(os.path.join(output_dir, 'imagesTr')))
        # print(num_training_cases)
        # file_extension = '.nii.gz'
        
        self.generate_dataset_json(file_ending=".nii.gz")
        
        
            
        # self.dataset_path
    
    def generate_dataset_json(self, 
                              labels: dict,
                              num_training_cases: int,
                              file_ending: str,
                              regions_class_order: Tuple[int, ...] = None,
                              dataset_name: str = None, reference: str = None, release: str = None, license: str = None,
                              description: str = None,
                              overwrite_image_reader_writer: str = None, **kwargs):

        has_regions: bool = any([isinstance(i, (tuple, list)) and len(i) > 1 for i in labels.values()])
        if has_regions:
            assert regions_class_order is not None, f"You have defined regions but regions_class_order is not set. " \
                                                    f"You need that."
        
        # prepare labels using label groups 
        # labels = {'background':0}
        
        
        # channel names need strings as keys
        channel_names = {k:n for k,n in enumerate(self.param["INPUT_MODALITIES"])}
        
        print(channel_names)
        
        keys = channel_names.keys()
        for k in keys:
            if not isinstance(k, str):
                channel_names[str(k)] = channel_names[k]
                del channel_names[k]

        # TODO: create a list with the labels and corresponding id values 

        # labels need ints as values
        for l in labels.keys():
            value = labels[l]
            if isinstance(value, (tuple, list)):
                value = tuple([int(i) for i in value])
                labels[l] = value
            else:
                labels[l] = int(labels[l])

        dataset_json = {
            'channel_names': channel_names, 
            'labels': labels,
            'numTraining': num_training_cases,
            'file_ending': file_ending,
        }

        if dataset_name is not None:
            dataset_json['name'] = dataset_name
        if reference is not None:
            dataset_json['reference'] = reference
        if release is not None:
            dataset_json['release'] = release
        if license is not None:
            dataset_json['licence'] = license
        if description is not None:
            dataset_json['description'] = description
        if overwrite_image_reader_writer is not None:
            dataset_json['overwrite_image_reader_writer'] = overwrite_image_reader_writer
        if regions_class_order is not None:
            dataset_json['regions_class_order'] = regions_class_order

        dataset_json.update(kwargs)

        save_json(dataset_json, join(self.dataset_path, 'dataset.json'), sort_keys=False)
                
    def __save_data(self, hf, data_type, label_groups):
        
        data_set = self.train_set if data_type == 'Tr' else self.test_set
        pb_desc = "Saving training data" if data_type == 'Tr' else "Saving test data"
        
        progress_bar = tqdm(data_set, desc=pb_desc)
                
        for pat in progress_bar:
            
            for m, modality in enumerate(self.params['INPUT_MODALITIES']):
                
                mod_data = hf[f'{pat}/modalities/{modality}'][:]
                voxel_spacing = [hf[f'{pat}/coordinates/dx'][()], hf[f'{pat}/coordinates/dy'][()], hf[f'{pat}/coordinates/dz'][()]]
                output_path = os.path.join(self.dataset_path, f'images{data_type}',  f"{data_type}Dat{int(pat):03d}_{m:04d}.nii.gz")
                self.save_as_nifti( np.transpose(mod_data, (2, 1, 0)), voxel_spacing,  output_path, modality=modality)
            
            label_data = np.zeros(mod_data.shape)
                            
            for lg in label_groups:
                for l in label_groups[lg]['labels']:
                    label_data[np.where(hf[f'{pat}/labels/{l}'][:]>0)] = label_groups[lg]['id']
            output_path = os.path.join(self.dataset_path, f'labels{data_type}',  f"{data_type}Dat{int(pat):03d}.nii.gz")
            self.save_as_nifti(np.transpose(label_data, (2, 1, 0)).astype(np.uint8), voxel_spacing, output_path, modality='RTSTRUCT')
            
dp = Preprocessing(params)
dp.prepare_data()#data_insight_file_path = "/home/ivazquez/Repos/nnUNet/raw_data/data_insight_Dataset001_NeckCTA.json")

  0%|          | 0/27 [00:00<?, ?it/s]

Saving training data:   0%|          | 0/24 [00:00<?, ?it/s]

Saving test data:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from typing import Tuple

from batchgenerators.utilities.file_and_folder_operations import save_json, join

output_dir = '../raw_data/Dataset001_NeckCTA'
channel_names = {0: 'CT'}
labels = {'common_carotid_artery_left': 1,
          'background': 0}
num_training_cases = len(os.listdir(os.path.join(output_dir, 'imagesTr')))
print(num_training_cases)
file_extension = '.nii.gz'

def generate_dataset_json(output_folder: str,
                          channel_names: dict,
                          labels: dict,
                          num_training_cases: int,
                          file_ending: str,
                          regions_class_order: Tuple[int, ...] = None,
                          dataset_name: str = None, reference: str = None, release: str = None, license: str = None,
                          description: str = None,
                          overwrite_image_reader_writer: str = None, **kwargs):
    """
    Generates a dataset.json file in the output folder

    channel_names:
        Channel names must map the index to the name of the channel, example:
        {
            0: 'T1',
            1: 'CT'
        }
        Note that the channel names may influence the normalization scheme!! Learn more in the documentation.

    labels:
        This will tell nnU-Net what labels to expect. Important: This will also determine whether you use region-based training or not.
        Example regular labels:
        {
            'background': 0,
            'left atrium': 1,
            'some other label': 2
        }
        Example region-based training:
        {
            'background': 0,
            'whole tumor': (1, 2, 3),
            'tumor core': (2, 3),
            'enhancing tumor': 3
        }

        Remember that nnU-Net expects consecutive values for labels! nnU-Net also expects 0 to be background!

    num_training_cases: is used to double check all cases are there!

    file_ending: needed for finding the files correctly. IMPORTANT! File endings must match between images and
    segmentations!

    dataset_name, reference, release, license, description: self-explanatory and not used by nnU-Net. Just for
    completeness and as a reminder that these would be great!

    overwrite_image_reader_writer: If you need a special IO class for your dataset you can derive it from
    BaseReaderWriter, place it into nnunet.imageio and reference it here by name

    kwargs: whatever you put here will be placed in the dataset.json as well

    """
    has_regions: bool = any([isinstance(i, (tuple, list)) and len(i) > 1 for i in labels.values()])
    if has_regions:
        assert regions_class_order is not None, f"You have defined regions but regions_class_order is not set. " \
                                                f"You need that."
    # channel names need strings as keys
    keys = list(channel_names.keys())
    for k in keys:
        if not isinstance(k, str):
            channel_names[str(k)] = channel_names[k]
            del channel_names[k]

    # labels need ints as values
    for l in labels.keys():
        value = labels[l]
        if isinstance(value, (tuple, list)):
            value = tuple([int(i) for i in value])
            labels[l] = value
        else:
            labels[l] = int(labels[l])

    dataset_json = {
        'channel_names': channel_names,  # previously this was called 'modality'. I didn't like this so this is
        # channel_names now. Live with it.
        'labels': labels,
        'numTraining': num_training_cases,
        'file_ending': file_ending,
    }

    if dataset_name is not None:
        dataset_json['name'] = dataset_name
    if reference is not None:
        dataset_json['reference'] = reference
    if release is not None:
        dataset_json['release'] = release
    if license is not None:
        dataset_json['licence'] = license
    if description is not None:
        dataset_json['description'] = description
    if overwrite_image_reader_writer is not None:
        dataset_json['overwrite_image_reader_writer'] = overwrite_image_reader_writer
    if regions_class_order is not None:
        dataset_json['regions_class_order'] = regions_class_order

    dataset_json.update(kwargs)

    save_json(dataset_json, join(output_folder, 'dataset.json'), sort_keys=False)
    print('here')
    
generate_dataset_json(output_dir, channel_names, labels, num_training_cases, file_extension)

In [ ]:
import h5py

with h5py.File('../raw_data/preprocessed_hdf5_files/preprocessed_Dataset001_NeckCTA.h5', 'r') as f:
    print(f.keys())
    print(f['1'].keys())

The scheme introduced above results in the following folder structure. Given is an example for the first Dataset of the MSD: BrainTumour. This dataset hat four input channels: FLAIR (0000), T1w (0001), T1gd (0002) and T2w (0003). Note that the imagesTs folder is optional and does not have to be present.

```
nnUNet_raw/Dataset001_BrainTumour/
├── dataset.json
├── imagesTr
│   ├── BRATS_001_0000.nii.gz
│   ├── BRATS_001_0001.nii.gz
│   ├── BRATS_001_0002.nii.gz
│   ├── BRATS_001_0003.nii.gz
│   ├── BRATS_002_0000.nii.gz
│   ├── BRATS_002_0001.nii.gz
│   ├── BRATS_002_0002.nii.gz
│   ├── BRATS_002_0003.nii.gz
│   ├── ...
├── imagesTs
│   ├── BRATS_485_0000.nii.gz
│   ├── BRATS_485_0001.nii.gz
│   ├── BRATS_485_0002.nii.gz
│   ├── BRATS_485_0003.nii.gz
│   ├── BRATS_486_0000.nii.gz
│   ├── BRATS_486_0001.nii.gz
│   ├── BRATS_486_0002.nii.gz
│   ├── BRATS_486_0003.nii.gz
│   ├── ...
└── labelsTr
    ├── BRATS_001.nii.gz
    ├── BRATS_002.nii.gz
    ├── ...
```

Example of data arrangement

```
nnUNet_raw/Dataset002_Heart/
├── dataset.json
├── imagesTr
│   ├── la_003_0000.nii.gz
│   ├── la_004_0000.nii.gz
│   ├── ...
├── imagesTs
│   ├── la_001_0000.nii.gz
│   ├── la_002_0000.nii.gz
│   ├── ...
└── labelsTr
    ├── la_003.nii.gz
    ├── la_004.nii.gz
    ├── ...
```


```json
{ 
 "channel_names": {  # formerly modalities
   "0": "T2", 
   "1": "ADC"
 }, 
 "labels": {  # THIS IS DIFFERENT NOW!
   "background": 0,
   "PZ": 1,
   "TZ": 2
 }, 
 "numTraining": 32, 
 "file_ending": ".nii.gz"
 "overwrite_image_reader_writer": "SimpleITKIO"  # optional! If not provided nnU-Net will automatically determine the ReaderWriter
 }
 ```
 The channel_names determine the normalization used by nnU-Net. If a channel is marked as 'CT', then a global normalization based on the intensities in the foreground pixels will be used. If it is something else, per-channel z-scoring will be used. Refer to the methods section in our paper for more details. nnU-Net v2 introduces a few more normalization schemes to choose from and allows you to define your own, see here for more information.

---
# Function to display CT and contours

In [ ]:
PAT_ID = 3
# DATA_DIRECTORY = "/home/ivazquez/Repos/nnUNet/raw_data/neck_cta/anonymized"
WINDOW_WIDTH = 400 # HU
WINDOW_CENTER = 40 # HU
INITIAL_SLICE = None
COLORMAP = 'gray'

dt = DicomToolbox(patient_data_directory = DATA_DIRECTORY)
dt.expected_data = ['ct', 'rtstruct']
dt.uniform_slice_thickness = False
dt.parse_dicom_files(PAT_ID, mask_resolution = 'ct', mask_names_only=False)

ct = dt.ct.data # grab the ct data
orig_coords = copy.deepcopy(dt.ct.coordinates) # grab the coordinates
# ct, dt.ct.coordinates = resize_to_user_resolution(ct, dt.ct.coordinates, [1,1,1]) 

ct = apply_windowing(ct, WINDOW_WIDTH, WINDOW_CENTER) # apply windowing
structures = [c for c in dt.contours]

# Create a Select widget
select_structures = Select(
    options=structures,
    value=structures[0],
    description='Structures:',
    rows=len(structures))

display(HBox([select_structures]))

In [ ]:
import tkinter as tk
from tkinter import ttk, Listbox, Scrollbar
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import copy
from dicom_tools import DicomToolbox  # Ensure this module is available

class DicomViewer:
    def __init__(self, root, data_directory, pat_id, window_width, window_center):
        self.root = root
        self.data_directory = data_directory
        self.pat_id = pat_id
        self.window_width = window_width
        self.window_center = window_center

        self.dt = DicomToolbox(patient_data_directory=data_directory)
        self.dt.expected_data = ['ct', 'rtstruct']
        self.dt.uniform_slice_thickness = False
        self.dt.parse_dicom_files(pat_id, mask_resolution='ct', mask_names_only=False)

        self.ct = self.dt.ct.data
        self.orig_coords = copy.deepcopy(self.dt.ct.coordinates)
        self.ct = self.apply_windowing(self.ct, window_width, window_center)
        self.structures = [c for c in self.dt.contours]

        self.create_widgets()
        self.update_plot()

    def create_widgets(self):
        self.structure_listbox = Listbox(self.root, selectmode=tk.SINGLE)
        for struct in self.structures:
            self.structure_listbox.insert(tk.END, struct)
        self.structure_listbox.bind('<<ListboxSelect>>', lambda event: self.update_plot())
        self.structure_listbox.pack(side=tk.LEFT, fill=tk.Y)

        scrollbar = Scrollbar(self.root)
        scrollbar.pack(side=tk.LEFT, fill=tk.Y)
        self.structure_listbox.config(yscrollcommand=scrollbar.set)
        scrollbar.config(command=self.structure_listbox.yview)

        self.figure, self.ax = plt.subplots(figsize=(7, 7))
        self.canvas = FigureCanvasTkAgg(self.figure, master=self.root)
        self.canvas.get_tk_widget().pack()

        self.slice_var = tk.IntVar(value=self.ct.shape[0] // 2)
        self.slice_slider = ttk.Scale(self.root, from_=0, to=self.ct.shape[0] - 1, orient="horizontal", variable=self.slice_var)
        self.slice_slider.pack(fill=tk.X)
        self.slice_slider.bind("<Motion>", lambda event: self.update_plot())

        self.root.bind('<Left>', self.previous_slice)
        self.root.bind('<Right>', self.next_slice)

    def apply_windowing(self, image, window_width, window_center):
        min_hu = window_center - window_width // 2
        max_hu = window_center + window_width // 2
        image = np.clip(image, min_hu, max_hu)
        image = (image - min_hu) / (max_hu - min_hu) * 255.0
        return image

    def update_plot(self, event=None):
        slice_idx = self.slice_var.get()
        selected_idx = self.structure_listbox.curselection()
        if selected_idx:
            structure_name = self.structure_listbox.get(selected_idx)
            structure = self.dt.contours[structure_name].data

            self.ax.clear()
            ct_extent = [self.dt.ct.coordinates.x.min(), self.dt.ct.coordinates.x.max(),
                         self.dt.ct.coordinates.y.max(), self.dt.ct.coordinates.y.min()]
            self.ax.imshow(self.ct[slice_idx], cmap='gray', interpolation='none', extent=ct_extent)
            x, y = self.dt.ct.coordinates.x, self.dt.ct.coordinates.y
            self.ax.contour(x, y, structure[slice_idx], levels=[0.5], colors='red')
            self.ax.axis('off')

            self.canvas.draw()

    def previous_slice(self, event):
        current_slice = self.slice_var.get()
        if current_slice > 0:
            self.slice_var.set(current_slice - 1)
            self.update_plot()

    def next_slice(self, event):
        current_slice = self.slice_var.get()
        if current_slice < self.ct.shape[0] - 1:
            self.slice_var.set(current_slice + 1)
            self.update_plot()

if __name__ == "__main__":
    root = tk.Tk()
    root.title("DICOM Viewer")

    data_directory = "/home/ivazquez/Documents/REPOS/nnUNet/raw_data/neck_cta/anonymized"
    pat_id = 1
    window_width = 400
    window_center = 40

    viewer = DicomViewer(root, data_directory, pat_id, window_width, window_center)
    root.mainloop()
